In [1]:
from solid import *
from solid.utils import *
import viewscad
from save_stl import save

i = 25.4
slices = 10

# makes a feature symmetric around (0,0)
def symmetry(feature, n=4):
    angles = [i*360/n for i in range(n)]
    return [rotate(i)(feature) for i in angles]

def key():
    return back(0.275/2*i)(left(0.194/2*i)(
        cube([0.194*i,0.275*i,0.275*i]) + forward((0.275-0.115)/2*i)(cube([0.25*i,0.115*i,0.275*i]))
    ))

def detent_wheel():
    wheel = up(0.135/2*i)(cylinder(r=0.65*i,h=.135/2*i)) + up(0.135*i)(key())
    cutout = forward(0.1*i)(right(0.1*i)(cube([1*i,1*i,1*i])))
    bump = forward(0.5*i)(up(0.135/2*i)(sphere(r=0.1*i, segments=slices) - up(0.55*i)(cube([i,i,i], center=True))))
    center_rod = down(2.5)(cylinder(d=0.1*i,h=0.25*i))
    return wheel - symmetry(cutout) + symmetry(bump) + center_rod

r=viewscad.Renderer()

In [2]:
# BGM111 PROGRAMMER:

pin_locations = {
    'swo1':[6.223,15.494],
    'swo':[8.382,13.335],
    'vdd':[6.35,9.398],
    'reset':[7.62,25.527],
    'gnd':[28.194,13.462]
}

def led_negative(mult=1.2):
    wire_tube = down(10)(rotate([90,0,0])(cylinder(d=5*mult,h=10)))
    return down(1.25)(cylinder(h=1.25,d=6.5*mult) + up(1.25)(cylinder(h=5,d=5.25*mult)) + up(6.25)(cylinder(h=15,d1=5.25*mult,d2=15*mult)) + down(10)(cylinder(h=10,d=5.25*mult))) + wire_tube

def pogo_pins(h=10):
    return union()([translate(pos+[0])(cylinder(h=h,d=1.2,segments=60)) for key,pos in pin_locations.items()])

def alignment_pins(h=10,mult=1):
    return union()([translate(pos+[0])(cylinder(h=h,d=2.5*mult)) for pos in [[2.5,2.5],[4,35],[30,35]]])

def pcb(h=10):
    outline = import_dxf("bt_board.dxf")
    return translate([2,2,0])(linear_extrude(height=h)(offset(r=0.25)(outline)))

def shell(h=14,bottom_thickness=1):
    return part()(cube([34,45,h]) - up(bottom_thickness)(hole()(pcb(h-bottom_thickness))))

def bottom():
    h=14
    bottom_thickness = h-12.5
    supports = [translate(pos+[0])(cylinder(h=15,d=5)) for pos in [[5,5],[20,20],[20,10]]]
    led = translate([10,40,h])(led_negative())
    return shell(h,bottom_thickness) + alignment_pins(h+2) - hole()(up(0.5)(pogo_pins(h))) - led

def top():
    h=6
    led = translate([10,40,0])(led_negative())
    return shell(h=h) - alignment_pins(mult=1.2) - pogo_pins() - translate([0,33,2])(cube([40,40,10])) - led
    
def holder():
    return top() + right(40)(bottom())

#r=viewscad.Renderer()
r.render(holder())
save(holder(), "pcb_holder.stl")

In [3]:
#STEERING LOCK MECHANISM

def n20_flip():
    return cube([9,10,24]) + cube([19.2,10,12]) + translate([9,5,18])(rotate([0,90,0])(cylinder(d=3,h=20)))

def profile():
    points = [[j*24.5 for j in i] for i in [[0.182,0],[.182,0.19],[0,0.3],[0,0.6],[.05,.6],[.05,.725],[0,.725],[0,1.05],[0.335,1.475],[1.435,1.475],[1.76,1.15],[1.76,.9],[1.435,.9],[1.435,.775],[1.385,.775],[1.385,.575],[1.435,.575],[1.435,.3],[1.2935,.19],[1.2935,0]]]
    return polygon(points)

def carrier():
    return cube([33,25,11])

#r.render(linear_extrude(5)(profile()))
#save(linear_extrude(5)(profile()), "throttle_lock_carrier.stl")